In [116]:
import os
import pandas as pd
import joblib
from tqdm import tqdm
import datetime
from datetime import datetime
from bs4 import BeautifulSoup

In [41]:
%%time

a = pd.DataFrame({'filename1': os.listdir()})
b = a.filename1.str.startswith("security")
files = list(a[b].filename1)
final_df = pd.DataFrame()

for file in tqdm(files):
    df = joblib.load(file)
    if final_df.empty:
        final_df = df
    else:
        final_df = pd.concat([df, final_df])

100%|██████████████████████████████████████████████████████████████████████████████████| 53/53 [00:37<00:00,  1.41it/s]

Wall time: 38 s


In [92]:
final_df.sample(2)

,title,summary_x,links,link,id,guidislink,published,published_parsed,title_detail.type,title_detail.language,...,authorposts,postcategory,customexcerpt,categories,date,URL,authors_y,keywords,summary_y,text
202,Securing open-source: how Google supports the ...,"<span class=""byline-author"">Posted by Maya Kac...","[{'rel': 'replies', 'type': 'application/atom+...",http://feedproxy.google.com/~r/GoogleOnlineSec...,"tag:blogger.com,1999:blog-1176949257541686127....",True,2020-01-14T09:00:00.000-08:00,"(2020, 1, 14, 17, 0, 0, 1, 14, 0)",text/plain,None,...,NaN,NaN,NaN,NaN,None,http://feedproxy.google.com/~r/GoogleOnlineSec...,"[Posted Maya Kaczorowski, Product Manager, Con...","[bug, securing, program, kubernetes, google, b...",What kind of bugs does the bounty program reco...,What kind of bugs does the bounty program reco...
101,Why DevSecOps Is Critical for Containers and K...,DevSecOps is a big and sometimes difficult shi...,"[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://www.darkreading.com/cloud/why-devsecop...,https://www.darkreading.com/cloud/why-devsecop...,False,"Fri, 08 May 2020 10:00:00 EDT","(2020, 5, 8, 14, 0, 0, 4, 129, 0)",text/plain,None,...,NaN,NaN,NaN,NaN,None,https://www.darkreading.com/cloud/why-devsecop...,[],"[devsecops, team, model, containers, critical,...",Why DevSecOps Is Critical for Containers and K...,Why DevSecOps Is Critical for Containers and K...


In [93]:
sum(list(final_df.URL == final_df.link))

19735

In [94]:
final_df2 = final_df[['title', 'summary_x', 'URL', 'published', 'keywords', 'summary_y', 'text' ]]

------

In [95]:
# the column 'published' has the time string.  Convert to datetime, as a timezone aware entry. 
# Fix EDT and PDT in advance

t1 = final_df2.published.apply(str)
t1 = [x.replace('EDT', '-0400') for x in t1]
t1 = [x.replace('PDT', '-0700') for x in t1]
t1 = [(pd.to_datetime(x)) for x in t1]
final_df2.published = t1

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [96]:
# Write a function to get just the data and time without timezone so it can be exported to Excel

def no_timezone(time_string):
    try:
        no_tz = datetime.strftime(time_string, "%Y-%m-%d %H:%M:%S")
    except:
        no_tz = 0
    return no_tz

In [97]:
# Get the published date (no timezone) in a new column
final_df2['published_date'] = final_df2.published.apply(no_timezone)

# Drop the published column 
final_df2.drop(columns = ['published'], inplace = True)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\user\Anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [98]:
len(final_df2)

19735

In [99]:
final_df2.sample(5)

,title,summary_x,URL,keywords,summary_y,text,published_date
320,5 risk management mistakes CISOs still make,<p>Corporate leaders now rank cybersecurity as...,https://www.csoonline.com/article/3538288/5-ri...,"[respondents, cyber, theyre, say, management, ...",Corporate leaders now rank cybersecurity as a ...,Corporate leaders now rank cybersecurity as a ...,2020-05-04 03:00:00
46,Hackers Exploit Critical Flaw in Ghost Platfor...,"Hackers targeted Ghost on Sunday, in a cryptoc...",https://threatpost.com/hackers-exploit-critica...,"[threatpost, exploit, servers, flaw, attack, p...",Hackers targeted the publishing platform Ghost...,Hackers targeted the publishing platform Ghost...,2020-05-04 19:23:41
248,Rapid Change is the New Normal,"<p style=""text-align: center;""><strong><span s...",http://feedproxy.google.com/~r/Securityweek/~3...,"[normal, rapid, networking, need, change, remo...",And given the rapid pace at which these change...,"Change is the New Normal, and it is Coming at ...",2020-04-23 16:24:25
148,Thoughts on OSSEC Con 2019,"<div class=""separator"" style=""clear: both; tex...",https://taosecurity.blogspot.com/2019/03/thoug...,"[thoughts, ossec, taosecurity, 2019]",,,2019-03-28 16:40:00
357,The global AI agenda: Asia-Pacific,"This report is part of “The global AI agenda,”...",https://www.technologyreview.com/2020/04/23/10...,"[agenda, main, organizations, challenges, usin...","This report is part of “The global AI agenda,”...","This report is part of “The global AI agenda,”...",2020-04-23 10:00:00


In [9]:
# print((final_df2.applymap(type) == list).all())

In [10]:
# print(final_df2.applymap(type))

In [11]:
# a = ['one', 'two', 'three']
# b = ','.join(a)

In [100]:

final_df2.keywords = [','.join(a) for a in final_df2.keywords.copy(deep=True)]

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [101]:
print('Length before drop_duplicates', len(final_df2))
final_df2 = final_df2.drop_duplicates()
print('Length after drop_duplicates', len(final_df2))


Length before drop_duplicates 19735
Length after drop_duplicates 18748


In [102]:
final_df2.text.nunique()
# final_df2.text2=final_df2.text

5153

In [103]:
final_df3 = final_df2.groupby('text').head(1)
print(len(final_df3), 'articles in final_df3')

5153 articles in final_df3


In [124]:
final_df3.reset_index(drop=True, inplace=True)

In [125]:
final_df3


,title,summary_x,URL,keywords,summary_y,text,published_date,corona
0,Friday Squid Blogging: Fishing for Jumbo Squid,Interesting article on the rise of the jumbo s...,https://www.schneier.com/blog/archives/2020/06...,"schneier,usual,squidinteresting,post,jumbo,res...",Friday Squid Blogging: Fishing for Jumbo Squid...,Friday Squid Blogging: Fishing for Jumbo Squid...,2020-06-26 20:57:09,False
1,The Unintended Harms of Cybersecurity,"Interesting research: ""Identifying Unintended ...",https://www.schneier.com/blog/archives/2020/06...,"services,schneier,user,preemptively,consequenc...","Interesting research: ""Identifying Unintended ...","Interesting research: ""Identifying Unintended ...",2020-06-26 12:00:59,False
2,Analyzing IoT Security Best Practices,"New research: ""Best Practices for IoT Security...",https://www.schneier.com/blog/archives/2020/06...,"schneier,practices,follow,guidelines,research,...",Analyzing IoT Security Best PracticesNew resea...,Analyzing IoT Security Best Practices\n\nNew r...,2020-06-25 12:09:36,False
3,COVID-19 Risks of Flying,"I fly a lot. Over the past five years, my aver...",https://www.schneier.com/blog/archives/2020/06...,"lot,schneier,trying,think,risks,mask,wearing,t...",This is all a prelude to saying that I have be...,COVID-19 Risks of Flying\n\nI fly a lot. Over ...,2020-06-24 17:32:30,True
4,Cryptocurrency Pump and Dump Scams,"Really interesting research: ""An examination o...",https://www.schneier.com/blog/archives/2020/06...,"schneier,pumps,discord,schemes,pump,cryptocurr...","Really interesting research: ""An examination o...","Really interesting research: ""An examination o...",2020-06-24 11:30:32,False
...,...,...,...,...,...,...,...,...
5148,Amazon confirms a major outbreak of COVID-19 a...,"<A HREF=""https://www.businessinsider.com/amazo...",http://www.techmeme.com/200423/p38#a200423p38,"video,past,security,warehouse,outbreak,surge,z...",— Zoom Video Communications Inc. has been lamb...,— Zoom Video Communications Inc. has been lamb...,2020-04-23 19:55:23,False
5149,The coronavirus is accelerating the shift to a...,"<A HREF=""https://www.ft.com/content/990e89de-8...",http://www.techmeme.com/200423/p33#a200423p33,"shift,financial,right,whiteboards,coronavirus,...",— Miro is a company in the right place at the ...,— Miro is a company in the right place at the ...,2020-04-23 17:30:02,False
5150,Doctors are using AI to triage covid-19 patien...,Rizwan Malik had always had an interest in AI....,https://www.technologyreview.com/2020/04/23/10...,"pandemic,health,system,triage,using,coronaviru...","The pandemic, in other words, has turned into ...",The Royal Bolton Hospital is among a growing n...,2020-04-23 14:00:00,True
5151,The race to save the first draft of coronaviru...,"Eight years ago, Suleika Jaouad was alone in a...",https://www.technologyreview.com/2020/04/21/99...,"save,pandemic,life,internet,history,coronaviru...",But getting the internet to archive as much as...,"Within a week, Blair’s tweet got the attention...",2020-04-21 09:00:00,True


In [126]:
final_df3.sample(3)

,title,summary_x,URL,keywords,summary_y,text,published_date,corona
3465,Trump threatens to take US out of WHO entirely...,"Trump letter blasts WHO's COVID-19 response, s...",https://arstechnica.com/?p=1677161,"funding,response,world,organization,threatens,...",President Trump yesterday threatened to perman...,President Trump yesterday threatened to perman...,2020-05-19 17:23:10,True
2982,Gears of war: When mechanical analog computers...,"From the archives: In some ways, new Navy comp...",https://arstechnica.com/?p=421869,"mechanical,digital,control,analog,system,syste...",We're talking about electro-mechanical analog ...,Update: We are resurfacing this feature from 2...,2020-05-24 11:46:47,False
937,India’s Milk Mantra secures $10M from US Inter...,"Milk Mantra, a startup that procures, packages...",http://feedproxy.google.com/~r/Techcrunch/~3/n...,"international,service,mantra,startup,india,mil...",U.S. International Development Finance Corpora...,"Milk Mantra, a startup that procures, packages...",2020-06-19 07:15:45,False


In [105]:
# sum(final_df3.title.str.contains("Corona|corona|coronavirus|covid|Covid|cyber"))

191

In [15]:
joblib.dump(final_df3, 'final_df3.pkl')

['final_df3.pkl']

In [137]:
final_df3.to_excel('temp.xlsx',index=False)

In [128]:
# Define a function for True/False for corona depending on if any of the corona related
# strings is found in a piece of text

def corona(text):
    corona_story_strings = ['covid', 'corona', 'sars', 'virus']
    return any(x in text for x in corona_story_strings)
    
    


In [129]:
final_df3['corona'] = final_df3.text.apply(corona)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [130]:
final_df3.shape

(5153, 8)

In [131]:
final_df3.corona.value_counts()

False    4037
True     1116
Name: corona, dtype: int64

In [132]:
final_df3.query('corona == True')

,title,summary_x,URL,keywords,summary_y,text,published_date,corona
3,COVID-19 Risks of Flying,"I fly a lot. Over the past five years, my aver...",https://www.schneier.com/blog/archives/2020/06...,"lot,schneier,trying,think,risks,mask,wearing,t...",This is all a prelude to saying that I have be...,COVID-19 Risks of Flying\n\nI fly a lot. Over ...,2020-06-24 17:32:30,True
24,France Télévisions group hit by a cyber attack...,<p>The France Télévisions group announced yest...,https://securityaffairs.co/wordpress/105269/ha...,"cyber,attack,antennas,impacted,télévisions,hit...",The France Télévisions group announced yesterd...,The France Télévisions group announced yesterd...,2020-06-27 12:11:34,True
30,"Microsoft Again Urges Exchange Server Patch, a...","Microsoft is once again… <a href=""https://heal...",https://healthitsecurity.com/news/microsoft-ag...,"organizations,urges,flaw,resurge,tools,servers...",By Jessica DavisMicrosoft is once again urging...,By Jessica Davis\n\nMicrosoft is once again ur...,0,True
33,3 Key Ways to Bolster Healthcare Cybersecurity...,"Throughout the course of… <a href=""https://hea...",https://healthitsecurity.com/news/3-key-ways-t...,"training,organizations,sector,devices,mfa,cybe...",While the number of reported cybersecurity inc...,By Jessica Davis\n\nThroughout the course of t...,0,True
35,DHS CISA: Serious Vulnerabilities Found in 6 M...,"The Department of Homeland… <a href=""https://h...",https://healthitsecurity.com/news/dhs-cisa-ser...,"medical,serious,cisa,dhs,vulnerabilities,devic...",If a hacker successfully exploited the flaw wi...,By Jessica Davis\n\nThe Department of Homeland...,0,True
...,...,...,...,...,...,...,...,...
5146,Nextdoor and Walmart partner on a new neighbor...,Neighborhood social network Nextdoor and Walma...,http://feedproxy.google.com/~r/Techcrunch/~3/_...,"offer,groups,feature,nextdoor,help,neighbors,n...",Neighborhood social network Nextdoor and Walma...,Neighborhood social network Nextdoor and Walma...,2020-04-23 16:22:09,True
5147,Daily Crunch: There’s a major iPhone email sec...,Apple plans to fix an iPhone email security bu...,http://feedproxy.google.com/~r/Techcrunch/~3/a...,"google,bug,daily,firm,security,extra,techcrunc...",Apple plans to fix an iPhone email security bu...,Apple plans to fix an iPhone email security bu...,2020-04-23 16:10:28,True
5150,Doctors are using AI to triage covid-19 patien...,Rizwan Malik had always had an interest in AI....,https://www.technologyreview.com/2020/04/23/10...,"pandemic,health,system,triage,using,coronaviru...","The pandemic, in other words, has turned into ...",The Royal Bolton Hospital is among a growing n...,2020-04-23 14:00:00,True
5151,The race to save the first draft of coronaviru...,"Eight years ago, Suleika Jaouad was alone in a...",https://www.technologyreview.com/2020/04/21/99...,"save,pandemic,life,internet,history,coronaviru...",But getting the internet to archive as much as...,"Within a week, Blair’s tweet got the attention...",2020-04-21 09:00:00,True


In [115]:
final_df3.query('corona == True').to_excel('temp.xlsx')